In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.33
session = InteractiveSession(config = config)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
label = pd.read_csv('undersampling_data.csv')
label

In [ ]:
img_data = []
error_list = []

for i in tqdm(range(label.shape[0])):
    try:
        img = image.load_img('img_black/' + str(label['img_path'][i]) + '.png', target_size=(256, 256, 1))
        img = image.img_to_array(img)
        img = img/255.
        img_data.append(img)
    except FileNotFoundError:
        error_list.append('img_black/' + str(label['img_path'][i])+ '.png')
        img = np.ones((256, 256, 3))
        img = img/255.
        img_data.append(img)

X = np.array(img_data)

In [ ]:
len(error_list)

In [ ]:
X.shape

In [ ]:
boolean_to_int = label.drop(['img_path', 'Unnamed: 0', 'labels'], axis=1)

boolean_to_int = boolean_to_int.astype(int)

boolean_to_int

In [ ]:
y = np.array(boolean_to_int)
y.shape

In [ ]:
i = 0
plt.imshow(X[i])
y[i]

In [ ]:
X_train, X_data, y_train, y_data = train_test_split(X, y, random_state=2045, test_size=0.2)

In [ ]:
X_valid, X_test, y_valid, y_test = train_test_split(X_data, y_data, random_state=2045, test_size=0.2)

In [ ]:
def create_model(optimizer='rmsprop'):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(256, 256,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='sigmoid'))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    
    return model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model)

optimizers = ['rmsprop', 'adam']
epochs = np.array([100, 200])
batches = np.array([16, 32, 64])
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

print(f'Best: {grid_result.best_score_:.5f} , using {grid_result.best_params_}')